In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, classification_report
from flask import Flask, render_template, request

app = Flask(__name__)

# Load the dataset (replace 'your_dataset.csv' with the actual dataset filename)
data = pd.read_csv('heart_data.csv')

# Data preprocessing
data['age'] = data['age'] / 365.25  # Convert age to years
data['cholesterol'] = data['cholesterol'] * 20  # Convert cholesterol to mg/dL
data = data.drop(['id', 'index'], axis=1)  # Drop unnecessary columns

# Split data into features (X) and target (y)
X = data.drop('cardio', axis=1)
y = data['cardio']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # Corrected line

# Initialize multiple classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Support Vector Machine": SVC(kernel='rbf', C=1, gamma='scale', probability=True)
}

# Train and evaluate multiple classifiers
for clf_name, clf in classifiers.items():
    clf.fit(X_train, y_train)

# Build an Artificial Neural Network (ANN) as before
model_ann = Sequential()
model_ann.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Increased hidden layers
model_ann.add(Dense(64, activation='relu'))
model_ann.add(Dense(32, activation='relu'))
model_ann.add(Dense(1, activation='sigmoid'))
model_ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ann.fit(X_train, y_train, epochs=150, batch_size=32)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Retrieve input values from the form
        age = float(request.form['age'])
        gender = float(request.form['gender'])
        height = float(request.form['height'])
        weight = float(request.form['weight'])
        ap_hi = float(request.form['ap_hi'])
        ap_lo = float(request.form['ap_lo'])
        cholesterol = float(request.form['cholesterol'])
        gluc = float(request.form['gluc'])
        smoke = float(request.form['smoke'])
        alco = float(request.form['alco'])
        active = float(request.form['active'])
        
        # Create an input feature vector
        new_data = np.array([age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke, alco, active]).reshape(1, -1)

        # Make predictions using multiple classifiers and ANN
        rf_prediction = classifiers["Random Forest"].predict(new_data)
        gb_prediction = classifiers["Gradient Boosting"].predict(new_data)
        svm_prediction = classifiers["Support Vector Machine"].predict(new_data)
        ann_prediction = (model_ann.predict(new_data) > 0.5).astype(int)

        # Determine the overall prediction (e.g., majority vote)
        overall_prediction = 1 if (rf_prediction[0] + gb_prediction[0] + svm_prediction[0] + ann_prediction[0]) >= 2 else 0

        # Display the prediction
        prediction_text = "likely to have heart disease" if overall_prediction == 1 else "not likely to have heart disease"
        return render_template('result.html', prediction=prediction_text)

if __name__ == '__main__':
    app.run(debug=True)


Epoch 1/150
1750/1750 [==============================] - 7s 3ms/step - loss: 0.5805 - accuracy: 0.7088
Epoch 2/150
1750/1750 [==============================] - 5s 3ms/step - loss: 0.5489 - accuracy: 0.7296
Epoch 3/150
1750/1750 [==============================] - 4s 2ms/step - loss: 0.5464 - accuracy: 0.7318
Epoch 4/150
1750/1750 [==============================] - 4s 2ms/step - loss: 0.5457 - accuracy: 0.7325
Epoch 5/150
1750/1750 [==============================] - 4s 2ms/step - loss: 0.5433 - accuracy: 0.7340
Epoch 6/150
1750/1750 [==============================] - 4s 2ms/step - loss: 0.5429 - accuracy: 0.7328
Epoch 7/150
1750/1750 [==============================] - 3s 2ms/step - loss: 0.5411 - accuracy: 0.7342
Epoch 8/150
1750/1750 [==============================] - 4s 2ms/step - loss: 0.5410 - accuracy: 0.7344
Epoch 9/150
1750/1750 [==============================] - 3s 2ms/step - loss: 0.5408 - accuracy: 0.7355
Epoch 10/150
1750/1750 [==============================] - 4s 2ms/step - l